
# 基于修改后的 MindQuantum 0.5.0 采用变分算法重构半导体双量子点单-三重态量子 CX 门

# 采用自定义的 MQLayer abs 来实现

In [3]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cx_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cx_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cx_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cx_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.1
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    if error <= error_min:
        error_min = error
        best_params = params
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-5:
        break
        
print('\nerror_min:', error_min, 'train_num:', j)
print(best_params)
final_state = [] 
for k in range(100): # 100 个测试点
    sim.reset()
    sim.set_qs(eval_x[k])
    sim.apply_circuit(circ, params)
    final_state.append(sim.get_qs())
error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
print(error)

error_min: 0.9759224573015861 error_now: 0.9759224573015861 train_num: 0
error_min: 0.17562114521099148 error_now: 0.32252924550205064 train_num: 100
error_min: 0.17562114521099148 error_now: 0.5113556279682479 train_num: 200
error_min: 0.17562114521099148 error_now: 0.2835904489734251 train_num: 300
error_min: 0.056214697159127414 error_now: 0.0778386433634688 train_num: 400
error_min: 0.056214697159127414 error_now: 0.1695953060475498 train_num: 500
error_min: 0.01515946423710246 error_now: 0.08913303802360395 train_num: 600
error_min: 0.01515946423710246 error_now: 0.07592769518683018 train_num: 700
error_min: 0.01515946423710246 error_now: 0.08441613530117908 train_num: 800
error_min: 0.01515946423710246 error_now: 0.33059912508380307 train_num: 900
error_min: 0.01515946423710246 error_now: 0.6034264952878733 train_num: 1000
error_min: 0.01515946423710246 error_now: 0.21981791348523383 train_num: 1100
error_min: 0.01515946423710246 error_now: 0.2825995534264468 train_num: 1200
erro

In [4]:
print('\nerror_min:', error_min, 'train_num:', j)
print(best_params)
final_state = []
for k in range(1000): # 100 个测试点
    sim.reset()
    sim.set_qs(eval_x[k])
    sim.apply_circuit(circ, best_params)
    final_state.append(sim.get_qs())
error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
print(error)


error_min: 0.012332183003952335 train_num: 6999
[3.0250957  0.76641613 1.9296288  2.313581   1.9758449  1.6931925
 0.22477806 0.46793172 0.4160759  2.5389414  0.22089939 0.7399326
 0.48215482 2.3717885  0.32094845 0.35486832 0.70596534 0.33416185
 0.25001308 2.4594276  0.06954821 2.041266   0.7886567  0.14270985
 0.14407785 0.13855346 0.20032802 0.8301053  0.9299462  0.48394054
 0.6397696  0.27576098 0.18415393 0.3055614  1.1686511  1.8513657
 4.899882   1.1090306  1.4938872  1.0060952  0.99469626 2.257587
 3.0448542  0.28859237]
0.012664642038423768


In [7]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cx_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cx_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cx_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cx_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.01
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
        j_tem = j
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_min: 0.9441378783013046 error_now: 0.9441378783013046 train_num: 0
error_min: 0.08822126361334559 error_now: 0.08822126361334559 train_num: 100
error_min: 0.03308202264383642 error_now: 0.04944551869846869 train_num: 200
error_min: 0.03308202264383642 error_now: 0.048101990076230394 train_num: 300
error_min: 0.02973382556355164 error_now: 0.035892231494172755 train_num: 400
error_min: 0.029488509496666593 error_now: 0.04442460177027041 train_num: 500
error_min: 0.029488509496666593 error_now: 0.046006425889988156 train_num: 600
error_min: 0.029488509496666593 error_now: 0.05798771247094192 train_num: 700
error_min: 0.029488509496666593 error_now: 0.10747602132408962 train_num: 800
error_min: 0.029488509496666593 error_now: 0.15724888727723974 train_num: 900
error_min: 0.029488509496666593 error_now: 0.07213183333880313 train_num: 1000
error_min: 0.029488509496666593 error_now: 0.04467560939258852 train_num: 1100
error_min: 0.026976519344692917 error_now: 0.026976519344692917 trai

NameError: name 'j_tem' is not defined

In [8]:
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_his: [0.9441378783013046, 0.9454587868943207, 0.9356518155073922, 0.9449791223055889, 0.9546330519847367, 0.9696222665411078, 0.9740294007008398, 0.9704941240941855, 0.972594457694587, 0.9755240502693913, 0.9788890432436265, 0.9823907953825196, 0.9884646070108094, 0.9891071903349443, 0.9873056469067465, 0.9826761242478198, 0.9757466914878566, 0.9677072237821865, 0.9609247799325458, 0.9539117839464462, 0.9606094171797647, 0.9662037560559291, 0.9665571399206943, 0.9607670778823814, 0.964798223138739, 0.9649073606899858, 0.9653049879791776, 0.9765219662845156, 0.9873486118798387, 0.9878779563329031, 0.9771170653300285, 0.9682444333559982, 0.9544063784612695, 0.9428894629879391, 0.9345193860201275, 0.9257670137524046, 0.9202623659916178, 0.9173497307744068, 0.9109202896607116, 0.9017153911212922, 0.8935802781306758, 0.8855326394306424, 0.8763636694511624, 0.8654967472755813, 0.8513774067657188, 0.8381456824939602, 0.8280050393633448, 0.8182223912329482, 0.8090501374250421, 0.7975083

In [ ]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cx_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cx_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cx_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cx_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.1
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
        j_tem = j
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)